In [1]:
from src.data_utils import load_and_preprocess, split_data
from src.next_token_dataset import NextTokenDataset
from src.lstm_model import LSTMAutocomplete
from src.lstm_train import LSTMTrainer
from src.eval_lstm import evaluate_rouge

from transformers import BertTokenizerFast

import torch
from torch.utils.data import Dataset, DataLoader

In [2]:
RANDOM_STATE = 42

SEQ_LEN = 20

Выполним загрузку и обработку текстовых данных, просмотрим первые строки:

In [3]:
df = load_and_preprocess("data/raw_dataset.txt")

df.head()

0    awww that s a bummer you shoulda got david car...
1    is upset that he can t update his facebook by ...
2    i dived many times for the ball managed to sav...
3       my whole body feels itchy and like its on fire
4    no it s not behaving at all i m mad why am i h...
Name: cleaned_text, dtype: object

Выполним разбиение на обучающую, тренировочную и валидационную выборки и выведем их размер:

In [4]:
train_texts, val_texts, test_texts = split_data(df, RANDOM_STATE)

display(len(df))
display(len(train_texts))
display(len(test_texts))
display(len(val_texts))

1601127

1280901

160113

160113

In [5]:
# загружаем токенизатор
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# тренировочный, тестовый и валидационный датасеты
train_dataset = NextTokenDataset(train_texts, tokenizer, seq_len=SEQ_LEN)
test_dataset = NextTokenDataset(test_texts, tokenizer, seq_len=SEQ_LEN)
val_dataset = NextTokenDataset(val_texts, tokenizer, seq_len=SEQ_LEN)

# даталоадеры
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64) 
test_loader = DataLoader(test_dataset, batch_size=64) 

In [ ]:
vocab_size = tokenizer.vocab_size

# создаем и обучаем модель
model = LSTMAutocomplete(vocab_size, hidden_dim=128, eos_token_id=tokenizer.sep_token_id)
trainer = LSTMTrainer(model, tokenizer, vocab_size)

trainer.fit(train_loader, val_loader, n_epochs=3)

Device: cuda
------------------------------


100%|██████████| 35296/35296 [04:48<00:00, 122.46it/s]


Epoch 1 | Train Loss: 5.223 | Val Loss: 5.185
------------------------------


100%|██████████| 4470/4470 [10:53<00:00,  6.84it/s]


Metrics
rouge1: 0.0657
rouge2: 0.0066
rougeL: 0.0643
rougeLsum: 0.0643


In [7]:
# сгенерируем несколько токенов
prefix = torch.tensor(tokenizer.encode("i want to", add_special_tokens=False))
generated_ids = model.generate(prefix, max_new_tokens=2)
print(tokenizer.decode(generated_ids.tolist()))

prefix = torch.tensor(tokenizer.encode("my body feels", add_special_tokens=False))
generated_ids = model.generate(prefix, max_new_tokens=2)
print(tokenizer.decode(generated_ids.tolist()))

prefix = torch.tensor(tokenizer.encode("you shoulda got", add_special_tokens=False))
generated_ids = model.generate(prefix, max_new_tokens=2)
print(tokenizer.decode(generated_ids.tolist()))

i want to go to
my body feels like i
you shoulda got a new
